# SocialAL Model

### Fit single alpha with decay model to data - multiple subjects

KLS 3.3.22
Project info: https://osf.io/b48n2/

Model modified from : Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

In [1]:
import sys
print(sys.version)
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import os
from decimal import *

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


In [2]:
def update_value(Prob):
    invest = [0,3,6,9]
    retain = [9-x for x in invest] #print ("Retain list is: ", retain)
    shared = [2*x for x in invest] #print ("Shared list is: ", shared)
    EV = [(retain[x] + Prob*shared[x]) for x in range(0,4)]
    return EV

def update_prob(recip, Prob, alpha):
    Prob = Prob + alpha*(recip-Prob)
    return Prob

def get_action_selection_prob(beta, EV, choice):
    actionProb = Decimal(np.exp(beta*EV[choice-1])/np.sum([np.exp(beta*x) for x in EV]))
    return actionProb

def get_action_selection_probs(beta, EV):
    actionProbs = [get_action_selection_prob(beta, EV, x) for x in range(1,5)]
    return actionProbs

def get_likelihood_action(params, data):
    alpha = params[0]
    beta = params[1]
    decay = params[2]
    
    # initialize variables
    prob = [0.5, 0.5, 0.5]
    ev = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    totalLLH = 0 
    for trial in range(0, len(data)):
        trustee = data['Stim_Sequence'][trial] # get trustee type
        choice = data['Choice'][trial] # get choice made by participant
        response = data['Trustee_Response'][trial] # get response from trustee
        
        # compute the probability of selecting each option for that trustee
        probs = get_action_selection_probs(beta, ev[trustee])
        
        # use the probability of the selection (choice-probability) to update log likelihood
        if choice != 0:
            cprob = probs[choice-1] #print(cprob, isinstance(cprob, float))
            #print(cprob)

            #add to cumulative log likelihood
            totalLLH += -(math.log(cprob))
        
            # update prob and value
            if choice != 1:
                prob[trustee] = update_prob(response, prob[trustee], alpha)
            ev[trustee] = update_value(prob[trustee])
            
        # decay prob and value for other possible trustees 
        options = [0, 1, 2]
        if choice == 2 or choice == 3 or choice == 4:
            options.pop(trustee)
        for x in options:
            prob[x] = prob[x] + decay * (0.5 - prob[x])
            ev[x] =  update_value(prob[x])       
    return totalLLH

def model_fit_once(data):
        # initialize free parameters with randomly chosen numbers
        alpha=random.uniform(0, 1)
        beta=random.uniform(0, 1)
        decay=random.uniform(0,1)
        params = [alpha, beta, decay]
        
        #results = minimize(get_likelihood_action, 
                           #params, args =(data), method='BFGS', options = {'maxiter': 10000, 'disp': False})
        results = minimize(get_likelihood_action, 
                       params, args =(data), bounds = [(0, 1), (1e-10, 100), (0, 1)], 
                       options = {'maxiter': 10000, 'disp': False})
        return results

def model_fit(data):
    
    tries = 100 #  number of tries to find the best-fit parameter
    lowestLLH = math.inf 
    bestFit = 'NA'
    
    for i in range(tries):
        
        # initialize free parameters with randomly chosen numbers
        alpha=random.uniform(0, 1)
        beta=random.uniform(0, 1)
        decay=random.uniform(0,1)
        params = [alpha, beta, decay]

        # trying different solvers in the minimize call...
        results = minimize(get_likelihood_action, 
                           params, args =(data), bounds = [(0, 1), (1e-10, 20), (0, 1)], 
                           options = {'maxiter': 10000, 'disp': False})
        if (lowestLLH > results['fun'] and results['success']== True):
            lowestLLH = results['fun']
            bestFit = results
            
    return bestFit

Load and Clean Data

In [3]:
files = os.listdir('../../data/modeling')

In [4]:
def load_and_clean(file):
    path = os.path.join('../../data/modeling', file)
    dt = pd.read_csv(path)
    # recode trial type into numbers for model
    def stims(trial_type):
        if trial_type == "Trustworthy":
            return 0
        elif trial_type == "Neutral":
            return 1
        elif trial_type == "Untrustworthy":
            return 2
    dt['Stim_Sequence'] = dt['trial_type'].apply(stims)
    # rename response_key to choice
    def choices(response_key):
        if response_key == 'None':
            return 0 
        else:
            return response_key  
    dt['Choice'] = dt['response_key'].apply(choices)
    dt['Choice'] = pd.to_numeric(dt['Choice'])
    # calculte the trustee response
    def resp(trial_earnings):
        if trial_earnings >= 12:
            return 1
        else:
            return 0
    dt['Trustee_Response'] = dt['trial_earnings'].apply(resp)
    data = dt[['Stim_Sequence','Choice', 'Trustee_Response']]
    return(data)

Fit Model

In [5]:
print(getcontext())
params = [model_fit(load_and_clean(file)) for file in files]

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])


In [6]:
params

[      fun: 46.686237847470736
  hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
       jac: array([ 3.57545105e-03, -2.21902496e-03, -7.10542736e-06])
   message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
      nfev: 256
       nit: 45
    status: 0
   success: True
         x: array([4.41472722e-03, 2.00000000e+01, 3.22068328e-02]),
       fun: 51.64479495954003
  hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
       jac: array([2.13162821e-06, 4.26325641e-06, 5.97529137e+00])
   message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
      nfev: 92
       nit: 15
    status: 0
   success: True
         x: array([0.21207528, 0.33599059, 0.        ]),
       fun: 60.99695188927512
  hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
       jac: array([0., 0., 0.])
   message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
      nfev: 8
       nit: 1
    status: 0
   success: True
         x: array([0.e+00, 1.e-10, 1.e+00]),
       fun: 61.44773372

In [7]:
params = pd.DataFrame(params)
params['id'] = [file[:-4] for file in files]
params['alpha'] = [params['x'][y][0] for y in range(len(params))]
params['beta'] = [params['x'][y][1] for y in range(len(params))]
params['decay'] = [params['x'][y][2] for y in range(len(params))]
params['-LLH'] = [params['fun'][y] for y in range(len(params))]
params = params[['id', 'alpha', 'beta', 'decay', '-LLH']]
print(params)   

          id     alpha          beta     decay       -LLH
0   sub-2013  0.004415  2.000000e+01  0.032207  46.686238
1   sub-2007  0.212075  3.359906e-01  0.000000  51.644795
2   sub-2006  0.000000  1.000000e-10  1.000000  60.996952
3   sub-2012  1.000000  1.499341e-01  0.621076  61.447734
4   sub-2004  0.173984  1.224196e+00  0.010163  35.031480
5   sub-2010  0.000000  1.000000e-10  0.423580  59.610658
6   sub-1040  0.000000  1.000000e-10  1.000000  60.996952
7   sub-2011  0.017395  2.000000e+01  0.620404  50.854942
8   sub-2005  0.000438  2.000000e+01  0.028970  59.331232
9   sub-2001  1.000000  3.844717e-01  0.070958  44.140218
10  sub-2015  0.591213  3.570583e-01  0.586744  60.996952
11  sub-2014  0.481137  5.472684e-01  0.000000  37.068169
12  sub-2016  0.601390  9.581328e-02  0.201561  59.610658
13  sub-2002  1.000000  8.479472e-02  0.285423  61.636159
14  sub-2003  0.005209  2.000000e+01  0.380069  55.884542
15  sub-1009  0.001175  2.000000e+01  0.000000  57.089928
16  sub-1021  

In [8]:
# save parameters in text file
params.to_csv(path_or_buf = '../../output/single_alpha_with_decay_model_params.csv', index = False)